In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
import gc
import matplotlib.pyplot as plt
from lightgbm import LGBMClassifier

['POS_CASH_balance.csv', 'bureau_balance.csv', 'application_train.csv', 'previous_application.csv', 'installments_payments.csv', 'credit_card_balance.csv', 'sample_submission.csv', 'application_test.csv', 'bureau.csv']


In [2]:
application_train = pd.read_csv("../input/application_train.csv")
application_test = pd.read_csv("../input/application_test.csv")

In [3]:
print(application_train.shape, application_test.shape)

(307511, 122) (48744, 121)


In [4]:
application_train["source"] = "train"
application_test["source"] = "test"

1)  Append train and test data

In [5]:
application = pd.concat([application_train, application_test], axis=0, ignore_index=True)
application.head()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,AMT_ANNUITY,AMT_CREDIT,AMT_GOODS_PRICE,AMT_INCOME_TOTAL,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_YEAR,APARTMENTS_AVG,APARTMENTS_MEDI,APARTMENTS_MODE,BASEMENTAREA_AVG,BASEMENTAREA_MEDI,BASEMENTAREA_MODE,CNT_CHILDREN,CNT_FAM_MEMBERS,CODE_GENDER,COMMONAREA_AVG,COMMONAREA_MEDI,COMMONAREA_MODE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_ID_PUBLISH,DAYS_LAST_PHONE_CHANGE,DAYS_REGISTRATION,DEF_30_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,ELEVATORS_AVG,ELEVATORS_MEDI,ELEVATORS_MODE,EMERGENCYSTATE_MODE,ENTRANCES_AVG,ENTRANCES_MEDI,ENTRANCES_MODE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,FLAG_CONT_MOBILE,...,LIVINGAPARTMENTS_MODE,LIVINGAREA_AVG,LIVINGAREA_MEDI,LIVINGAREA_MODE,NAME_CONTRACT_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,NAME_INCOME_TYPE,NAME_TYPE_SUITE,NONLIVINGAPARTMENTS_AVG,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_AVG,NONLIVINGAREA_MEDI,NONLIVINGAREA_MODE,OBS_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,OCCUPATION_TYPE,ORGANIZATION_TYPE,OWN_CAR_AGE,REGION_POPULATION_RELATIVE,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,SK_ID_CURR,TARGET,TOTALAREA_MODE,WALLSMATERIAL_MODE,WEEKDAY_APPR_PROCESS_START,YEARS_BEGINEXPLUATATION_AVG,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_AVG,YEARS_BUILD_MEDI,YEARS_BUILD_MODE,source
0,24700.5,406597.5,351000.0,202500.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0247,0.0250,0.0252,0.0369,0.0369,0.0383,0,1.0,M,0.0143,0.0144,0.0144,-9461,-637,-2120,-1134.0,-3648.0,2.0,2.0,0.00,0.00,0.0000,No,0.0690,0.0690,0.0690,0.083037,0.262949,0.139376,1,...,0.022,0.0190,0.0193,0.0198,Cash loans,Secondary / secondary special,Single / not married,House / apartment,Working,Unaccompanied,0.0000,0.0000,0.0,0.0000,0.00,0.0,2.0,2.0,Laborers,Business Entity Type 3,NaN,0.018801,2,2,0,0,0,0,100002,1.0,0.0149,"Stone, brick",WEDNESDAY,0.9722,0.9722,0.9722,0.6192,0.6243,0.6341,train
1,35698.5,1293502.5,1129500.0,270000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0959,0.0968,0.0924,0.0529,0.0529,0.0538,0,2.0,F,0.0605,0.0608,0.0497,-16765,-1188,-291,-828.0,-1186.0,0.0,0.0,0.08,0.08,0.0806,No,0.0345,0.0345,0.0345,0.311267,0.622246,NaN,1,...,0.079,0.0549,0.0558,0.0554,Cash loans,Higher education,Married,House / apartment,State servant,Family,0.0039,0.0039,0.0,0.0098,0.01,0.0,1.0,1.0,Core staff,School,NaN,0.003541,1,1,0,0,0,0,100003,0.0,0.0714,Block,MONDAY,0.9851,0.9851,0.9851,0.7960,0.7987,0.8040,train
2,6750.0,135000.0,135000.0,67500.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0,1.0,M,NaN,NaN,NaN,-19046,-225,-2531,-815.0,-4260.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.555912,0.729567,1,...,NaN,NaN,NaN,NaN,Revolving loans,Secondary / secondary special,Single / not married,House / apartment,Working,Unaccompanied,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,Laborers,Government,26.0,0.010032,2,2,0,0,0,0,100004,0.0,NaN,NaN,MONDAY,NaN,NaN,NaN,NaN,NaN,NaN,train
3,29686.5,312682.5,297000.0,135000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2.0,F,NaN,NaN,NaN,-19005,-3039,-2437,-617.0,-9833.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.650442,NaN,1,...,NaN,NaN,NaN,NaN,Cash loans,Secondary / secondary special,Civil marriage,House / apartment,Working,Unaccompanied,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,Laborers,Business Entity Type 3,NaN,0.008019,2,2,0,0,0,0,100006,0.0,NaN,NaN,WEDNESDAY,NaN,NaN,NaN,NaN,NaN,NaN,train
4,21865.5,513000.0,513000.0,121500.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0,1.0,M,NaN,NaN,NaN,-19932,-3038,-3458,-1106.0,-4311.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.322738,NaN,1,...,NaN,NaN,NaN,NaN,Cash loans,Secondary / secondary special,Single / not married,House / apartment,Working,Unaccompanied,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,Core staff,Religion,NaN,0.028663,2,2,0,1,0,0,100007,0.0,NaN,NaN,THURSDAY,NaN,NaN,NaN,NaN,NaN,NaN,train


 2) Find columns with a lot of missing data and remove some of them

In [6]:
missing_cols_prcnt = application_train.isnull().sum()/application_train.shape[0] * 100
high_missing_values = missing_cols_prcnt[missing_cols_prcnt > 50]
high_missing_values_index = high_missing_values.index.tolist()
high_missing_values[:5]

OWN_CAR_AGE         65.990810
EXT_SOURCE_1        56.381073
APARTMENTS_AVG      50.749729
BASEMENTAREA_AVG    58.515956
YEARS_BUILD_AVG     66.497784
dtype: float64

In [7]:
correlations = application_train.corr()['TARGET'].sort_values()
corr_missing_cols = correlations.reindex(high_missing_values_index).sort_values()
missing_cols_to_be_dropped = corr_missing_cols.index.difference(corr_missing_cols[(corr_missing_cols > 0.02) | (corr_missing_cols < -0.02)].index).tolist()
len(missing_cols_to_be_dropped)

19

In [8]:
missing_cols_to_be_dropped

['BASEMENTAREA_MODE',
 'COMMONAREA_AVG',
 'COMMONAREA_MEDI',
 'COMMONAREA_MODE',
 'ENTRANCES_AVG',
 'ENTRANCES_MEDI',
 'ENTRANCES_MODE',
 'FONDKAPREMONT_MODE',
 'HOUSETYPE_MODE',
 'LANDAREA_AVG',
 'LANDAREA_MEDI',
 'LANDAREA_MODE',
 'NONLIVINGAPARTMENTS_AVG',
 'NONLIVINGAPARTMENTS_MEDI',
 'NONLIVINGAPARTMENTS_MODE',
 'NONLIVINGAREA_AVG',
 'NONLIVINGAREA_MEDI',
 'NONLIVINGAREA_MODE',
 'WALLSMATERIAL_MODE']

In [9]:
application = application.drop(missing_cols_to_be_dropped, axis=1)
application.shape

(356255, 104)

3) Separate numerical and categorical data

In [10]:
#Make sure 'TARGET' and 'Source' are excluded
categorical_vars = [f for f in application.columns if f!='TARGET' and f!='source' and application[f].dtype=="object"]
categorical_vars

['CODE_GENDER',
 'EMERGENCYSTATE_MODE',
 'FLAG_OWN_CAR',
 'FLAG_OWN_REALTY',
 'NAME_CONTRACT_TYPE',
 'NAME_EDUCATION_TYPE',
 'NAME_FAMILY_STATUS',
 'NAME_HOUSING_TYPE',
 'NAME_INCOME_TYPE',
 'NAME_TYPE_SUITE',
 'OCCUPATION_TYPE',
 'ORGANIZATION_TYPE',
 'WEEKDAY_APPR_PROCESS_START']

In [11]:
application[categorical_vars].head()

,CODE_GENDER,EMERGENCYSTATE_MODE,FLAG_OWN_CAR,FLAG_OWN_REALTY,NAME_CONTRACT_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,NAME_INCOME_TYPE,NAME_TYPE_SUITE,OCCUPATION_TYPE,ORGANIZATION_TYPE,WEEKDAY_APPR_PROCESS_START
0,M,No,N,Y,Cash loans,Secondary / secondary special,Single / not married,House / apartment,Working,Unaccompanied,Laborers,Business Entity Type 3,WEDNESDAY
1,F,No,N,N,Cash loans,Higher education,Married,House / apartment,State servant,Family,Core staff,School,MONDAY
2,M,NaN,Y,Y,Revolving loans,Secondary / secondary special,Single / not married,House / apartment,Working,Unaccompanied,Laborers,Government,MONDAY
3,F,NaN,N,Y,Cash loans,Secondary / secondary special,Civil marriage,House / apartment,Working,Unaccompanied,Laborers,Business Entity Type 3,WEDNESDAY
4,M,NaN,N,Y,Cash loans,Secondary / secondary special,Single / not married,House / apartment,Working,Unaccompanied,Core staff,Religion,THURSDAY


In [12]:
#Check once more that all numerical columns are indeed continuous and not discrete
numerical_vars = [f for f in application.columns if f!='TARGET' and f!='source' and application[f].dtype!="object"]
numerical_vars

['AMT_ANNUITY',
 'AMT_CREDIT',
 'AMT_GOODS_PRICE',
 'AMT_INCOME_TOTAL',
 'AMT_REQ_CREDIT_BUREAU_DAY',
 'AMT_REQ_CREDIT_BUREAU_HOUR',
 'AMT_REQ_CREDIT_BUREAU_MON',
 'AMT_REQ_CREDIT_BUREAU_QRT',
 'AMT_REQ_CREDIT_BUREAU_WEEK',
 'AMT_REQ_CREDIT_BUREAU_YEAR',
 'APARTMENTS_AVG',
 'APARTMENTS_MEDI',
 'APARTMENTS_MODE',
 'BASEMENTAREA_AVG',
 'BASEMENTAREA_MEDI',
 'CNT_CHILDREN',
 'CNT_FAM_MEMBERS',
 'DAYS_BIRTH',
 'DAYS_EMPLOYED',
 'DAYS_ID_PUBLISH',
 'DAYS_LAST_PHONE_CHANGE',
 'DAYS_REGISTRATION',
 'DEF_30_CNT_SOCIAL_CIRCLE',
 'DEF_60_CNT_SOCIAL_CIRCLE',
 'ELEVATORS_AVG',
 'ELEVATORS_MEDI',
 'ELEVATORS_MODE',
 'EXT_SOURCE_1',
 'EXT_SOURCE_2',
 'EXT_SOURCE_3',
 'FLAG_CONT_MOBILE',
 'FLAG_DOCUMENT_10',
 'FLAG_DOCUMENT_11',
 'FLAG_DOCUMENT_12',
 'FLAG_DOCUMENT_13',
 'FLAG_DOCUMENT_14',
 'FLAG_DOCUMENT_15',
 'FLAG_DOCUMENT_16',
 'FLAG_DOCUMENT_17',
 'FLAG_DOCUMENT_18',
 'FLAG_DOCUMENT_19',
 'FLAG_DOCUMENT_2',
 'FLAG_DOCUMENT_20',
 'FLAG_DOCUMENT_21',
 'FLAG_DOCUMENT_3',
 'FLAG_DOCUMENT_4',
 'FLAG

4) DAYS_EMPLOYED discrepancy

In [13]:
application["DAYS_EMPLOYED_ANOMALY"] = 0
application["DAYS_EMPLOYED_ANOMALY"].loc[application[application["DAYS_EMPLOYED"] > 0]["DAYS_EMPLOYED"].index] = 1
application["DAYS_EMPLOYED"].loc[application[application["DAYS_EMPLOYED"] > 0]["DAYS_EMPLOYED"].index] = 0
application[["DAYS_EMPLOYED", "DAYS_EMPLOYED_ANOMALY"]].head()

/opt/conda/lib/python3.6/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


,DAYS_EMPLOYED,DAYS_EMPLOYED_ANOMALY
0,-637,0
1,-1188,0
2,-225,0
3,-3039,0
4,-3038,0


5) XNA represents null in categiorical cols

In [14]:
xna_cols = []
for c in categorical_vars:
    if application[c][application[c]=='XNA'].shape[0] > 0:
        xna_cols.append(c)
        
for c in xna_cols:
    application[c] = application[c].replace('XNA', np.nan)

6)Amt credit missing values imputation

In [15]:
application[["AMT_CREDIT", "AMT_GOODS_PRICE"]].describe()

,AMT_CREDIT,AMT_GOODS_PRICE
count,3.562550e+05,3.559770e+05
mean,5.877674e+05,5.280200e+05
std,3.986237e+05,3.660650e+05
min,4.500000e+04,4.050000e+04
25%,2.700000e+05,2.340000e+05
50%,5.002110e+05,4.500000e+05
75%,7.975575e+05,6.750000e+05
max,4.050000e+06,4.050000e+06


In [16]:
application[["AMT_CREDIT", "AMT_GOODS_PRICE"]].isnull().sum()

AMT_CREDIT           0
AMT_GOODS_PRICE    278
dtype: int64

In [17]:
missing_goods_price_indices = application[application["AMT_GOODS_PRICE"].isnull()].index
norm_factor = np.mean(application["AMT_GOODS_PRICE"])/np.mean(application["AMT_CREDIT"])

In [18]:
#Replacing the missing values of AMT_GOODS_PRICE
application.loc[missing_goods_price_indices, "AMT_GOODS_PRICE"] = application.loc[missing_goods_price_indices, "AMT_CREDIT"] * norm_factor

In [19]:
application[["AMT_CREDIT", "AMT_GOODS_PRICE"]].isnull().sum()

AMT_CREDIT         0
AMT_GOODS_PRICE    0
dtype: int64

7) For other numerical columns impute missing values by median(including ext_source_3)

In [20]:
numeric_cols_with_missing_vals = application[numerical_vars].isnull().sum()[application[numerical_vars].isnull().sum() > 0].index.tolist()
categorical_cols_with_missing_vals = application[categorical_vars].isnull().sum()[application[categorical_vars].isnull().sum() > 0].index.tolist()

In [21]:
numeric_cols_with_missing_vals[:5]

['AMT_ANNUITY',
 'AMT_REQ_CREDIT_BUREAU_DAY',
 'AMT_REQ_CREDIT_BUREAU_HOUR',
 'AMT_REQ_CREDIT_BUREAU_MON',
 'AMT_REQ_CREDIT_BUREAU_QRT']

In [22]:
#np.median(application["AMT_ANNUITY"].dropna())
for f in numeric_cols_with_missing_vals:
    application[f] = application[f].replace(np.nan, np.median(application[f].dropna()))

In [23]:
application[numerical_vars].isnull().sum()[application[numerical_vars].isnull().sum() > 0]

Series([], dtype: int64)

8) For categorical columns, impute missing values by mode

In [24]:
categorical_cols_with_missing_vals

['CODE_GENDER',
 'EMERGENCYSTATE_MODE',
 'NAME_TYPE_SUITE',
 'OCCUPATION_TYPE',
 'ORGANIZATION_TYPE']

In [25]:
# application["CODE_GENDER"].value_counts().idxmax()
for f in categorical_cols_with_missing_vals:
    max_count = application[f].value_counts().idxmax()
    application[f] = application[f].fillna(max_count)

In [26]:
application[categorical_vars].isnull().sum()[application[categorical_vars].isnull().sum() > 0]

Series([], dtype: int64)

9) If possible add the domain specific columns as mentioned in other kernels

In [27]:
application['CREDIT_INCOME_PERCENT'] = application['AMT_CREDIT'] / application['AMT_INCOME_TOTAL']
application['ANNUITY_INCOME_PERCENT'] = application['AMT_ANNUITY'] / application['AMT_INCOME_TOTAL']
application['CREDIT_TERM'] = application['AMT_ANNUITY'] / application['AMT_CREDIT']
application['DAYS_EMPLOYED_PERCENT'] = application['DAYS_EMPLOYED'] / application['DAYS_BIRTH']

10) One hot encoding of categorical variables

In [28]:
# dummies = pd.get_dummies(train_test.loc[:, dummy_colnames])
# train_test = pd.concat([train_test, dummies], axis=1)

dummies = pd.get_dummies(application[categorical_vars])
application = pd.concat([application, dummies], axis=1)
application = application.drop(categorical_vars, axis=1)
application.shape

(356255, 220)

In [29]:
application.head()

,AMT_ANNUITY,AMT_CREDIT,AMT_GOODS_PRICE,AMT_INCOME_TOTAL,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_YEAR,APARTMENTS_AVG,APARTMENTS_MEDI,APARTMENTS_MODE,BASEMENTAREA_AVG,BASEMENTAREA_MEDI,CNT_CHILDREN,CNT_FAM_MEMBERS,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_ID_PUBLISH,DAYS_LAST_PHONE_CHANGE,DAYS_REGISTRATION,DEF_30_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,ELEVATORS_AVG,ELEVATORS_MEDI,ELEVATORS_MODE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,FLAG_CONT_MOBILE,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,...,ORGANIZATION_TYPE_Industry: type 7,ORGANIZATION_TYPE_Industry: type 8,ORGANIZATION_TYPE_Industry: type 9,ORGANIZATION_TYPE_Insurance,ORGANIZATION_TYPE_Kindergarten,ORGANIZATION_TYPE_Legal Services,ORGANIZATION_TYPE_Medicine,ORGANIZATION_TYPE_Military,ORGANIZATION_TYPE_Mobile,ORGANIZATION_TYPE_Other,ORGANIZATION_TYPE_Police,ORGANIZATION_TYPE_Postal,ORGANIZATION_TYPE_Realtor,ORGANIZATION_TYPE_Religion,ORGANIZATION_TYPE_Restaurant,ORGANIZATION_TYPE_School,ORGANIZATION_TYPE_Security,ORGANIZATION_TYPE_Security Ministries,ORGANIZATION_TYPE_Self-employed,ORGANIZATION_TYPE_Services,ORGANIZATION_TYPE_Telecom,ORGANIZATION_TYPE_Trade: type 1,ORGANIZATION_TYPE_Trade: type 2,ORGANIZATION_TYPE_Trade: type 3,ORGANIZATION_TYPE_Trade: type 4,ORGANIZATION_TYPE_Trade: type 5,ORGANIZATION_TYPE_Trade: type 6,ORGANIZATION_TYPE_Trade: type 7,ORGANIZATION_TYPE_Transport: type 1,ORGANIZATION_TYPE_Transport: type 2,ORGANIZATION_TYPE_Transport: type 3,ORGANIZATION_TYPE_Transport: type 4,ORGANIZATION_TYPE_University,WEEKDAY_APPR_PROCESS_START_FRIDAY,WEEKDAY_APPR_PROCESS_START_MONDAY,WEEKDAY_APPR_PROCESS_START_SATURDAY,WEEKDAY_APPR_PROCESS_START_SUNDAY,WEEKDAY_APPR_PROCESS_START_THURSDAY,WEEKDAY_APPR_PROCESS_START_TUESDAY,WEEKDAY_APPR_PROCESS_START_WEDNESDAY
0,24700.5,406597.5,351000.0,202500.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0247,0.0250,0.0252,0.0369,0.0369,0,1.0,-9461,-637,-2120,-1134.0,-3648.0,2.0,2.0,0.00,0.00,0.0000,0.083037,0.262949,0.139376,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,35698.5,1293502.5,1129500.0,270000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0959,0.0968,0.0924,0.0529,0.0529,0,2.0,-16765,-1188,-291,-828.0,-1186.0,0.0,0.0,0.08,0.08,0.0806,0.311267,0.622246,0.533482,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2,6750.0,135000.0,135000.0,67500.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0880,0.0874,0.0840,0.0765,0.0761,0,1.0,-19046,-225,-2531,-815.0,-4260.0,0.0,0.0,0.00,0.00,0.0000,0.506155,0.555912,0.729567,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,29686.5,312682.5,297000.0,135000.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0880,0.0874,0.0840,0.0765,0.0761,0,2.0,-19005,-3039,-2437,-617.0,-9833.0,0.0,0.0,0.00,0.00,0.0000,0.506155,0.650442,0.533482,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,21865.5,513000.0,513000.0,121500.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0880,0.0874,0.0840,0.0765,0.0761,0,1.0,-19932,-3038,-3458,-1106.0,-4311.0,0.0,0.0,0.00,0.00,0.0000,0.506155,0.322738,0.533482,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [30]:
application[application.TARGET == 0].shape[0]/ application[application.TARGET == 1].shape[0]

11.387150050352467

11) Imbalanced classes

In [31]:
train = application[application["source"] == 'train']
test = application[application["source"] == 'test']

In [32]:
X_train = train.drop(["source", "TARGET"], axis=1)
Y_train = train["TARGET"]
X_test = test.drop(["source", "TARGET"], axis=1)

In [33]:
X_test.shape, X_train.shape

((48744, 218), (307511, 218))

In [34]:
default_index = np.array(train[train.TARGET == 1].index)
repaid_index = np.array(train[train.TARGET == 0].index)
repaid_permuted_index = np.random.permutation(repaid_index)[:default_index.shape[0]*3]

balanced_indices = np.concatenate((repaid_permuted_index, default_index))
balanced_permuted_indices = np.random.permutation(balanced_indices)

balanced_permuted_indices
Y_balanced = train.TARGET[balanced_permuted_indices].values
X_balanced_data = train.iloc[balanced_permuted_indices, :].drop(['TARGET', 'source'], axis=1)

In [35]:
X_balanced_data.head()

,AMT_ANNUITY,AMT_CREDIT,AMT_GOODS_PRICE,AMT_INCOME_TOTAL,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_YEAR,APARTMENTS_AVG,APARTMENTS_MEDI,APARTMENTS_MODE,BASEMENTAREA_AVG,BASEMENTAREA_MEDI,CNT_CHILDREN,CNT_FAM_MEMBERS,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_ID_PUBLISH,DAYS_LAST_PHONE_CHANGE,DAYS_REGISTRATION,DEF_30_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,ELEVATORS_AVG,ELEVATORS_MEDI,ELEVATORS_MODE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,FLAG_CONT_MOBILE,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,...,ORGANIZATION_TYPE_Industry: type 7,ORGANIZATION_TYPE_Industry: type 8,ORGANIZATION_TYPE_Industry: type 9,ORGANIZATION_TYPE_Insurance,ORGANIZATION_TYPE_Kindergarten,ORGANIZATION_TYPE_Legal Services,ORGANIZATION_TYPE_Medicine,ORGANIZATION_TYPE_Military,ORGANIZATION_TYPE_Mobile,ORGANIZATION_TYPE_Other,ORGANIZATION_TYPE_Police,ORGANIZATION_TYPE_Postal,ORGANIZATION_TYPE_Realtor,ORGANIZATION_TYPE_Religion,ORGANIZATION_TYPE_Restaurant,ORGANIZATION_TYPE_School,ORGANIZATION_TYPE_Security,ORGANIZATION_TYPE_Security Ministries,ORGANIZATION_TYPE_Self-employed,ORGANIZATION_TYPE_Services,ORGANIZATION_TYPE_Telecom,ORGANIZATION_TYPE_Trade: type 1,ORGANIZATION_TYPE_Trade: type 2,ORGANIZATION_TYPE_Trade: type 3,ORGANIZATION_TYPE_Trade: type 4,ORGANIZATION_TYPE_Trade: type 5,ORGANIZATION_TYPE_Trade: type 6,ORGANIZATION_TYPE_Trade: type 7,ORGANIZATION_TYPE_Transport: type 1,ORGANIZATION_TYPE_Transport: type 2,ORGANIZATION_TYPE_Transport: type 3,ORGANIZATION_TYPE_Transport: type 4,ORGANIZATION_TYPE_University,WEEKDAY_APPR_PROCESS_START_FRIDAY,WEEKDAY_APPR_PROCESS_START_MONDAY,WEEKDAY_APPR_PROCESS_START_SATURDAY,WEEKDAY_APPR_PROCESS_START_SUNDAY,WEEKDAY_APPR_PROCESS_START_THURSDAY,WEEKDAY_APPR_PROCESS_START_TUESDAY,WEEKDAY_APPR_PROCESS_START_WEDNESDAY
143519,19435.5,288873.0,238500.0,135000.0,0.0,0.0,0.0,0.0,0.0,1.0,0.088,0.0874,0.084,0.0765,0.0761,1,3.0,-12912,-137,-4762,-1671.0,-2129.0,0.0,0.0,0.0,0.0,0.0,0.506155,0.449491,0.533482,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
65794,29047.5,787086.0,657000.0,157500.0,0.0,0.0,0.0,0.0,0.0,6.0,0.088,0.0874,0.084,0.0765,0.0761,0,1.0,-13870,-1726,-4504,-1280.0,-3253.0,0.0,0.0,0.0,0.0,0.0,0.506155,0.695131,0.146442,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
35008,72607.5,2085120.0,1800000.0,310500.0,0.0,0.0,0.0,0.0,0.0,0.0,0.088,0.0874,0.084,0.0765,0.0761,0,1.0,-21505,-210,-3668,-1540.0,-12.0,0.0,0.0,0.0,0.0,0.0,0.509829,0.343984,0.382502,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
115769,8757.0,135000.0,135000.0,67500.0,0.0,0.0,0.0,0.0,0.0,1.0,0.088,0.0874,0.084,0.0765,0.0761,0,1.0,-11755,-1123,-3238,-2016.0,-5811.0,0.0,0.0,0.0,0.0,0.0,0.506155,0.285898,0.533482,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
218735,26217.0,808650.0,675000.0,90000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.088,0.0874,0.084,0.0765,0.0761,1,3.0,-21559,0,-4628,0.0,-3182.0,0.0,0.0,0.0,0.0,0.0,0.506155,0.735407,0.544235,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


12) LGBM K FOLD <br>
* Initially I tried Logistic regression, logistic regression with regularization, random forest and lgboost all on created balanced dataset.   
* Logistic regression with regularization was nearly as good as random forest but both of these techniques were outperformed by lgboost
* We then ran a LGBOOST model on the entire dataset to give a score of 0.766

In [36]:
k_fold = KFold(n_splits = 5, shuffle = True, random_state = 50)
x_train = np.array(X_train.values)
y_train = np.array(Y_train.values)
x_test = np.array(X_test.values)
test_predictions = np.zeros(x_test.shape[0])
train_auc = []
valid_auc = []
for train_indices, valid_indices in k_fold.split(x_train):
    train_data, train_target = x_train[train_indices], y_train[train_indices]
    valid_data, valid_target = x_train[valid_indices], y_train[valid_indices]
    clf = LGBMClassifier(
        n_estimators=1000,
        learning_rate=0.03,
        num_leaves=30,
        colsample_bytree=.8,
        subsample=.9,
        max_depth=7,
        reg_alpha=.1,
        reg_lambda=.1,
        min_split_gain=.01,
        min_child_weight=2,
        silent=-1,
        verbose=-1,
    )
        
    clf.fit(train_data, train_target, 
        eval_set= [(train_data, train_target), (valid_data, valid_target)], 
        eval_names = ['train', 'valid'],
        eval_metric='auc', verbose=100, early_stopping_rounds=100  #30
   )
    
    #best_iteration = clf.best_iteration_
    
    valid_score = clf.best_score_['valid']['auc']
    train_score = clf.best_score_['train']['auc']
    valid_auc.append(valid_score)
    train_auc.append(train_score)
    
    test_predictions += clf.predict_proba(x_test, num_iteration=clf.best_iteration_)[:,1]/k_fold.n_splits
    
    gc.enable()
    del clf, train_data, valid_data
    gc.disable()
        

Training until validation scores don't improve for 100 rounds.
[100]	train's auc: 0.765628	valid's auc: 0.749967
[200]	train's auc: 0.784126	valid's auc: 0.758826
[300]	train's auc: 0.795859	valid's auc: 0.761739
[400]	train's auc: 0.805169	valid's auc: 0.762723
[500]	train's auc: 0.813458	valid's auc: 0.763335
[600]	train's auc: 0.821048	valid's auc: 0.763871
[700]	train's auc: 0.828631	valid's auc: 0.764265
[800]	train's auc: 0.835744	valid's auc: 0.764236
Early stopping, best iteration is:
[720]	train's auc: 0.829966	valid's auc: 0.764321
Training until validation scores don't improve for 100 rounds.
[100]	train's auc: 0.765286	valid's auc: 0.751578
[200]	train's auc: 0.782691	valid's auc: 0.76052
[300]	train's auc: 0.794726	valid's auc: 0.764456
[400]	train's auc: 0.804131	valid's auc: 0.765908
[500]	train's auc: 0.812472	valid's auc: 0.766313
[600]	train's auc: 0.820416	valid's auc: 0.767053
[700]	train's auc: 0.827635	valid's auc: 0.767409
Early stopping, best iteration is:
[679]

In [37]:
kfold_auc = pd.DataFrame({'train_auc': train_auc, 
                          'valid_auc': valid_auc})
kfold_auc

,train_auc,valid_auc
0,0.829966,0.764321
1,0.826232,0.767441
2,0.826040,0.772256
3,0.837132,0.766504
4,0.847014,0.767416


In [38]:
test_predictions.shape

(48744,)

In [39]:
test_output = pd.DataFrame({'SK_ID_CURR':application_test.SK_ID_CURR.values, 'TARGET':test_predictions})
test_output.head()

,SK_ID_CURR,TARGET
0,100001,0.029940
1,100005,0.098959
2,100013,0.013503
3,100028,0.046132
4,100038,0.135582


The code below has all been commented as it was just in experimentation phase. However, it has good details on implementing logistic regression, grid search, cross validation score and random forest

13) Logistic Regression

In [ ]:
# %%time
# lr_cv = cross_val_score(LogisticRegression(), X_balanced_data, Y_balanced, cv=5, scoring='roc_auc')

In [ ]:
# np.mean(lr_cv)

Logistic regression on upsampled data is giving a score of 0.633 by Cross validation. 
Let's run a grid search to find the optimal parameters of Penalty and lambda

In [ ]:
# %%time
# hyperparameters = {
#     'penalty': ['l1', 'l2'], 
#     'C': np.logspace(0,1,5)}

# clf = GridSearchCV(LogisticRegression(), hyperparameters, cv=5, verbose=0)
# best_fit = clf.fit(X_balanced_data, Y_balanced)

In [ ]:
# best_fit.best_params_

In [ ]:
# best_fit.best_score_

In [ ]:
#lr_regularized_cv = cross_val_score(LogisticRegression(penalty='l1', C=1), X_balanced_data, Y_balanced, cv=5, scoring='roc_auc')

In [ ]:
#np.mean(lr_regularized_cv)

In [ ]:
# %%time
# lr_regularized = LogisticRegression(penalty='l1', C=1).fit(X_balanced_data, Y_balanced)

In [ ]:
# Y_predicted = lr_regularized.predict_proba(X_train)[:, 1]
# print(roc_auc_score(Y_train, Y_predicted))
# print(lr_regularized.score(X_train, Y_train))

In [ ]:
# Y_pred_test = lr_regularized.predict_proba(X_test)[:, 1]
# test_output = pd.DataFrame({'SK_ID_CURR':application_test.SK_ID_CURR.values, 'TARGET':Y_pred_test})

In [ ]:
# test_output.head()

In [ ]:
# test_output.to_csv('regularized_logistic_output.csv', index = False)

14) Random Forest

In [ ]:
# %%time
# random_forest_cv = cross_val_score(RandomForestClassifier(n_estimators=600, min_samples_split=30, min_samples_leaf = 10, max_features="auto"), 
#                                    X_balanced_data, 
#                                    Y_balanced, 
#                                    cv=5, 
#                                    scoring='roc_auc')

In [ ]:
# np.mean(random_forest_cv)

In [ ]:
# %%time
# rf_model = RandomForestClassifier(n_estimators=600, min_samples_split=30, min_samples_leaf = 10, max_features="auto")
# rf_model = rf_model.fit(X_balanced_data, Y_balanced)
# Y_predicted = rf_model.predict_proba(X_train)[:, 1]
# print(roc_auc_score(Y_train, Y_predicted))

In [ ]:
# Y_pred_test = rf_model.predict_proba(X_test)[:, 1]
# test_output = pd.DataFrame({'SK_ID_CURR':application_test.SK_ID_CURR.values, 'TARGET':Y_pred_test})
# test_output.head()

In [ ]:
# test_output.to_csv('random_forest_output.csv', index = False)

15)  Next up - Gradient Boosting

In [ ]:
# from lightgbm import LGBMClassifier

# clf = LGBMClassifier(
#         n_estimators=1000,
#         learning_rate=0.03,
#         num_leaves=30,
#         colsample_bytree=.8,
#         subsample=.9,
#         max_depth=7,
#         reg_alpha=.1,
#         reg_lambda=.1,
#         min_split_gain=.01,
#         min_child_weight=2,
#         silent=-1,
#         verbose=-1,
#     )
        
# clf.fit(X_balanced_data, Y_balanced, 
#         eval_set= [(X_balanced_data, Y_balanced), (X_train, Y_train)], 
#         eval_metric='auc', verbose=100, early_stopping_rounds=100  #30
#        )
        

In [ ]:
# feature_importance_df = pd.DataFrame({'features': X_train.columns.tolist(), 
#                                       'feature_importance': clf.feature_importances_})
# feature_importance_df = feature_importance_df.sort_values('feature_importance', ascending=False)
# feature_importance_df.head(10)

In [ ]:
# top_10 = feature_importance_df[:10]
# # n = list(range(0,10))
# # plt.bar(top_10.features, top_10.feature_importance)
# # plt.xticks(rotation = 90)
# # plt.show()
# fig, ax = plt.subplots(figsize=(15,5))
# pos = list(range(0,10))
# plt.bar(pos, top_10.feature_importance.values, width = 0.5, align='center')
# ax.set_xticks(pos)
# ax.set_xticklabels(top_10.features.values, rotation= 90)
# # plt.xticks(pos, (inc_typ_idx)) - substitute of above 2 lines
# plt.show()

In [ ]:
# subsetted_features = feature_importance_df[feature_importance_df.feature_importance != 0]["features"].tolist()
# X_balanced = X_balanced_data[subsetted_features]
# clf = LGBMClassifier(
#         n_estimators=2000,
#         learning_rate=0.03,
#         num_leaves=30,
#         colsample_bytree=.8,
#         subsample=.9,
#         max_depth=7,
#         reg_alpha=.1,
#         reg_lambda=.1,
#         min_split_gain=.01,
#         min_child_weight=2,
#         silent=-1,
#         verbose=-1,
#     )
        
# clf.fit(X_balanced, Y_balanced, 
#         eval_set= [(X_balanced, Y_balanced), (X_train[subsetted_features], Y_train)], 
#         eval_metric='auc', verbose=100, early_stopping_rounds=100  #30
#        )
        

In [ ]:
# test_predictions = np.zeros(X_test[subsetted_features].shape[0])
# test_predictions += clf.predict_proba(X_test[subsetted_features], num_iteration=clf.best_iteration_)[:,1]

In [ ]:
# test_predictions
# test_output = pd.DataFrame({'SK_ID_CURR':application_test.SK_ID_CURR.values, 'TARGET':test_predictions})
# test_output.head()

In [ ]:
test_output.to_csv('light_gbm_v3_feature_imp.csv', index = False)